In [50]:
import numpy as np
import struct
import math
import os
from scipy.io import FortranFile
from types import SimpleNamespace
from astropy.io import ascii
from config import *
import const

from functions import move_to_sim_dir, get_info


In [53]:
sim_round = 9
sim_name = 'fiducial'
dump = 54

move_to_sim_dir(sim_round, sim_name)
info = get_info(dump)
os.chdir("output_{dump:05d}".format(dump=dump))


Moving to directory '/home/za9132/scratch/romain/round9/fiducial'.


In [70]:
filename = "{filetype}_{dump:05d}.txt"

with open(filename.format(filetype='info', dump=dump)) as f:
    content = f.readlines()

info = SimpleNamespace()
for line in content:
    line_sp = line.split("=")
    if len(line_sp) > 1:
        try:
            setattr(info, line_sp[0].strip(), eval(line_sp[1].strip()))
        except NameError:
            setattr(info, line_sp[0].strip(), line_sp[1].strip())
            

            

In [ ]:
# Read the number of variables from the hydro_file_descriptor.txt
hydrofile = infile+"/hydro_file_descriptor.txt"
list_vars, _ = read_descriptor(hydrofile)

# Store the total number of hydro variables
info["nvar"] = len(list_vars)

# Make sure we always read the coordinates
list_vars.extend(("level","x","y","z","dx"))
nvar_read = len(list_vars)

# Now read the amr and hydro files =============================================
# We have to open the files in binary format, and count all the bytes in the ===
# file structure to extract just the data we need. =============================
# See output_amr.f90 and output_hydro.f90 in the RAMSES source. ================
print("Processing %i files in " % (info["ncpu"]) + infile)

# We will store the cells in a dictionary which we build as we go along.
# The final concatenation into a single array will be done once at the end.
data_pieces = dict()
npieces = 0

# Allocate work arrays
twotondim = 2**info["ndim"]
xcent = np.zeros([8,3],dtype=np.float64)
xg    = np.zeros([info["ngridmax"],3],dtype=np.float64)
son   = np.zeros([info["ngridmax"],twotondim],dtype=np.int32)
var   = np.zeros([info["ngridmax"],twotondim,nvar_read],dtype=np.float64)
xyz   = np.zeros([info["ngridmax"],twotondim,info["ndim"]],dtype=np.float64)
ref   = np.zeros([info["ngridmax"],twotondim],dtype=bool) # np.bool removed for numpy>=1.24

partinfofile = infile+"/header_"+infile.split("_")[-1]+".txt"
info["particle_count"] = {}
try:
    _lines = open(partinfofile).readlines()[1:-2]
    Nparttot = 0
    for line in _lines:
        part_type, _tmp = line.split()
        part_count = int(_tmp)
        info["particle_count"][part_type] = part_count
        Nparttot += part_count
    info["particle_count"]["total"] = Nparttot

    particle_vars, particle_dtypes = read_descriptor(infile + "/part_file_descriptor.txt")
except FileNotFoundError:
    info["particle_count"]["total"] = 0
    particle_vars, particle_dtypes = []
npart_var = len(particle_vars)
npart_read = 0

part_data = np.zeros([info["particle_count"]["total"], npart_var], dtype=np.float64)

iprog = 1
istep = 10
ncells_tot = 0

# Loop over the cpus and read the AMR and HYDRO files in binary format
for k in range(info["ncpu"]):

    # Print progress
    percentage = int(float(k)*100.0/float(info["ncpu"]))
    if percentage >= iprog*istep:
        print("%3i%% : read %10i cells" % (percentage,ncells_tot))
        iprog += 1

    # Read binary AMR file
    amr_fname = generate_fname(nout,ftype="amr",cpuid=k+1)
    with open(amr_fname, mode='rb') as amr_file: # b is important -> binary
        amrContent = amr_file.read()

    # Read binary HYDRO file
    hydro_fname = generate_fname(nout,ftype="hydro",cpuid=k+1)
    with open(hydro_fname, mode='rb') as hydro_file: # b is important -> binary
        hydroContent = hydro_file.read()


    # Need to extract info from the file header on the first loop
    if k == 0:

        # nx,ny,nz
        ninteg = 2
        nfloat = 0
        nlines = 2
        nstrin = 0
        nquadr = 0
        offset = 4*ninteg + 8*(nlines+nfloat) + nstrin + nquadr*16 + 4
        [nx,ny,nz] = struct.unpack("3i", amrContent[offset:offset+12])
        ncoarse = nx*ny*nz
        xbound = [float(int(nx/2)),float(int(ny/2)),float(int(nz/2))]

        # nboundary
        ninteg = 7
        nfloat = 0
        nlines = 5
        nstrin = 0
        nquadr = 0
        offset = 4*ninteg + 8*(nlines+nfloat) + nstrin + nquadr*16 + 4
        nboundary = struct.unpack("i", amrContent[offset:offset+4])[0]
        ngridlevel = np.zeros([info["ncpu"]+nboundary,info["levelmax"]],dtype=np.int32)

        # noutput
        ninteg = 9
        nfloat = 1
        nlines = 8
        nstrin = 0
        nquadr = 0
        offset = 4*ninteg + 8*(nlines+nfloat) + nstrin + nquadr*16 + 4
        noutput = struct.unpack("i", amrContent[offset:offset+4])[0]

        # dtold, dtnew
        ninteg = 12
        nfloat = 2+2*noutput
        nlines = 12
        nstrin = 0
        nquadr = 0
        offset = 4*ninteg + 8*(nlines+nfloat) + nstrin + nquadr*16 + 4
        info["dtold"] = struct.unpack("%id"%info["levelmax"], amrContent[offset:offset+8*info["levelmax"]])

        # info["dtold"] = eng.get_binary_data(fmt="%id"%(self.info["levelmax"]),\
                             # content=amrContent,ninteg=ninteg,nlines=nlines,nfloat=nfloat)
        nfloat += 1 + info["levelmax"]
        nlines += 1
        offset = 4*ninteg + 8*(nlines+nfloat) + nstrin + nquadr*16 + 4
        info["dtnew"] = struct.unpack("%id"%info["levelmax"], amrContent[offset:offset+8*info["levelmax"]])
        # info["dtnew"] = eng.get_binary_data(fmt="%id"%(self.info["levelmax"]),\
        #                      content=amrContent,ninteg=ninteg,nlines=nlines,nfloat=nfloat)

    # Read the number of grids
    ninteg = 14+(2*info["ncpu"]*info["levelmax"])
    nfloat = 18+(2*noutput)+(2*info["levelmax"])
    nlines = 21
    nstrin = 0
    nquadr = 0
    offset = 4*ninteg + 8*(nlines+nfloat) + nstrin + nquadr*16 + 4
    ngridlevel[:info["ncpu"],:] = np.asarray(struct.unpack("%ii"%(info["ncpu"]*info["levelmax"]), amrContent[offset:offset+4*info["ncpu"]*info["levelmax"]])).reshape(info["levelmax"],info["ncpu"]).T

    # Read boundary grids if any
    if nboundary > 0:
        ninteg = 14+(3*info["ncpu"]*info["levelmax"])+(10*info["levelmax"])+(2*nboundary*info["levelmax"])
        nfloat = 18+(2*noutput)+(2*info["levelmax"])
        nlines = 25
        nstrin = 0
        nquadr = 0
        offset = 4*ninteg + 8*(nlines+nfloat) + nstrin + nquadr*16 + 4
        ngridlevel[info["ncpu"]:info["ncpu"]+nboundary,:] = np.asarray(struct.unpack("%ii"%(nboundary*info["levelmax"]), amrContent[offset:offset+4*nboundary*info["levelmax"]])).reshape(info["levelmax"],nboundary).T

    # Determine bound key precision
    ninteg = 14+(3*info["ncpu"]*info["levelmax"])+(10*info["levelmax"])+(3*nboundary*info["levelmax"])+5
    nfloat = 18+(2*noutput)+(2*info["levelmax"])
    nlines = 21+2+3*min(1,nboundary)+1+1
    nstrin = 128
    nquadr = 0
    offset = 4*ninteg + 8*(nlines+nfloat) + nstrin + nquadr*16
    key_size = struct.unpack("i", amrContent[offset:offset+4])[0]

    # Offset for AMR
    ninteg1 = 14+(3*info["ncpu"]*info["levelmax"])+(10*info["levelmax"])+(3*nboundary*info["levelmax"])+5+3*ncoarse
    nfloat1 = 18+(2*noutput)+(2*info["levelmax"])
    nlines1 = 21+2+3*min(1,nboundary)+1+1+1+3
    nstrin1 = 128 + key_size

    # Offset for HYDRO
    ninteg2 = 5
    nfloat2 = 1
    nlines2 = 6
    nstrin2 = 0

    # Loop over levels
    for ilevel in range(info["levelmax"]):

        # Geometry
        dxcell=0.5**(ilevel+1)
        dx2=0.5*dxcell
        for ind in range(twotondim):
            iz=int((ind)/4)
            iy=int((ind-4*iz)/2)
            ix=int((ind-2*iy-4*iz))
            xcent[ind,0]=(float(ix)-0.5)*dxcell
            xcent[ind,1]=(float(iy)-0.5)*dxcell
            xcent[ind,2]=(float(iz)-0.5)*dxcell

        # Cumulative offsets in AMR file
        ninteg_amr = ninteg1
        nfloat_amr = nfloat1
        nlines_amr = nlines1
        nstrin_amr = nstrin1

        # Cumulative offsets in HYDRO file
        ninteg_hydro = ninteg2
        nfloat_hydro = nfloat2
        nlines_hydro = nlines2
        nstrin_hydro = nstrin2

        # Loop over domains
        for j in range(nboundary+info["ncpu"]):

            ncache = ngridlevel[j,ilevel]

            # Skip two lines of integers
            nlines_hydro += 2
            ninteg_hydro += 2

            if ncache > 0:

                if j == k:
                    # xg: grid coordinates
                    ninteg = ninteg_amr + ncache*3
                    nfloat = nfloat_amr
                    nlines = nlines_amr + 3
                    nstrin = nstrin_amr
                    for n in range(info["ndim"]):
                        offset = 4*ninteg + 8*(nlines+nfloat+n*(ncache+1)) + nstrin + 4
                        xg[:ncache,n] = struct.unpack("%id"%(ncache), amrContent[offset:offset+8*ncache])

                    # son indices
                    ninteg = ninteg_amr + ncache*(4+2*info["ndim"])
                    nfloat = nfloat_amr + ncache*info["ndim"]
                    nlines = nlines_amr + 4 + 3*info["ndim"]
                    nstrin = nstrin_amr
                    for ind in range(twotondim):
                        offset = 4*(ninteg+ind*ncache) + 8*(nlines+nfloat+ind) + nstrin + 4
                        son[:ncache,ind] = struct.unpack("%ii"%(ncache), amrContent[offset:offset+4*ncache])
                        # var: hydro variables
                        #jvar = 0
                        for ivar in range(info["nvar"]):
                            #if var_read[ivar]:
                            offset = 4*ninteg_hydro + 8*(nlines_hydro+nfloat_hydro+(ind*info["nvar"]+ivar)*(ncache+1)) + nstrin_hydro + 4
                            var[:ncache,ind,ivar] = struct.unpack("%id"%(ncache), hydroContent[offset:offset+8*ncache])
                            #jvar += 1
                        var[:ncache,ind,-5] = float(ilevel+1)
                        for n in range(info["ndim"]):
                            xyz[:ncache,ind,n] = xg[:ncache,n] + xcent[ind,n]-xbound[n]
                            var[:ncache,ind,-4+n] = xyz[:ncache,ind,n]*info["boxlen"]
                        var[:ncache,ind,-1] = dxcell*info["boxlen"]
                        # ref: True if the cell is unrefined
                        ref[:ncache,ind] = np.logical_not(np.logical_and(son[:ncache,ind] > 0,ilevel < info["levelmax"]-1))

                    cube = np.where(ref[:ncache,:])
                    cells = var[cube]
                    ncells = np.shape(cells)[0]
                    if ncells > 0:
                        ncells_tot += ncells
                        npieces += 1
                        # Add the cells in the master dictionary
                        data_pieces["piece"+str(npieces)] = cells

                # Now increment the offsets while looping through the domains
                ninteg_amr += ncache*(4+3*twotondim+2*info["ndim"])
                nfloat_amr += ncache*info["ndim"]
                nlines_amr += 4 + 3*twotondim + 3*info["ndim"]

                nfloat_hydro += ncache*twotondim*info["nvar"]
                nlines_hydro += twotondim*info["nvar"]

        # Now increment the offsets while looping through the levels
        ninteg1 = ninteg_amr
        nfloat1 = nfloat_amr
        nlines1 = nlines_amr
        nstrin1 = nstrin_amr

        ninteg2 = ninteg_hydro
        nfloat2 = nfloat_hydro
        nlines2 = nlines_hydro
        nstrin2 = nstrin_hydro

    # Read binary particle file
    if info["particle_count"]["total"] > 0:
        part_fname = generate_fname(nout,ftype="part",cpuid=k+1)
        with open(part_fname, mode='rb') as part_file:
            partContent = part_file.read()
        npart, = struct.unpack("i", partContent[28:32])
        pcounts = info["particle_count"]
        has_tracers = any(v for k, v in pcounts.items() if k.endswith("_tracer"))
        # Offset to "mstar_tot"
        offset = 72
        if has_tracers:
            offset += struct.calcsize("4i")  # tracer_seed

        # Read mstar, mstar_lost
        mstar, = struct.unpack("d", partContent[offset+4:offset+12])
        offset += 4+8+4  # mstar
        mstar_lost, = struct.unpack("d", partContent[offset+4:offset+12])
        offset += 4+8+4  # mstar_lost
        offset += 4+4+4  # nsink
        offset += 4      # jump to beginning of record

        for ivar, var_dtype in enumerate(particle_dtypes):
            s = struct.calcsize(var_dtype)
            endPos = offset + s * npart
            part_data[npart_read:npart_read+npart, ivar] = struct.unpack("%s%s" % (npart, var_dtype), partContent[offset:endPos])
            offset = endPos + 8

        npart_read += npart
    else:
        mstar = mstar_lost = np.nan

# Merge all the data pieces into the master data array
master_data_array = np.concatenate(list(data_pieces.values()), axis=0)

# Free memory
del data_pieces,xcent,xg,son,var,xyz,ref


print("Total number of cells loaded: %i" % ncells_tot)
if npart_read > 0:
    print("Total particles loaded: %i" % npart_read)

# This is the master data dictionary.
data = {"data": {}, "info": info, "sinks": {"nsinks": 0}, "stellars": {"nstellars": 0}}
for i in range(len(list_vars)):
    theKey = list_vars[i]
    data["data"][theKey] = master_data_array[:,i]

if npart_read > 0:
    data["particle"] = {}
    for ivar, var_name in enumerate(particle_vars):
        data["particle"][var_name] = part_data[:, ivar].copy()

del part_data

data["info"]["mstar"] = mstar
data["info"]["mstar_lost"] = mstar_lost

# Append useful variables to dictionary
data["data"]["unit_d"] = info["unit_d"]
data["data"]["unit_l"] = info["unit_l"]
data["data"]["unit_t"] = info["unit_t"]
data["data"]["boxlen"] = info["boxlen"]
data["data"]["ncells"] = ncells_tot
data["data"]["time"  ] = info["time"]

# Read sink particles if present
sinkfile = infile+"/sink_"+infile.split("_")[-1]+".csv"
try:
    with open(sinkfile) as f:
        content = f.readlines()
    # Read the file header to get information on fields
    sink_vars = content[0].rstrip().replace(" # ", "").split(",")
    sink_units = content[1].rstrip().replace(" # ", "").split(",")
    data["sinks"]["nsinks"] = len(content) - 2
    if data["sinks"]["nsinks"] > 0:
        # sinks = dict()
        for entry in sink_vars:
            data["sinks"][entry] = np.zeros(data["sinks"]["nsinks"], dtype=np.float64)
        for i in range(data["sinks"]["nsinks"]):
            # line = np.asarray(content[i+2].rstrip().split(","), dtype=np.float64)
            line = content[i+2].rstrip().split(",")
            for j, entry in enumerate(sink_vars):
                # Try to convert to float
                try:
                    data["sinks"][entry][i] = np.float64(line[j])
                except ValueError:
                    data["sinks"][entry][i] = np.nan
        data["sinks"]["id"] = np.int32(data["sinks"]["id"])
        data["sinks"]["level"] = np.int32(data["sinks"]["level"])
except IOError:
    pass

# Read stellar particles if present
stellarfile = infile+"/stellar_"+infile.split("_")[-1]+".csv"
try:
    with open(stellarfile) as f:
        content = f.readlines()
    # Read the file header to get information on fields
    stellar_vars = content[0].rstrip().replace(" # ", "").split(",")
    stellar_units = content[1].rstrip().replace(" # ", "").split(",")
    data["stellars"]["nstellars"] = len(content) - 2
    if data["stellars"]["nstellars"] > 0:
        for entry in stellar_vars:
            data["stellars"][entry] = np.zeros(data["stellars"]["nstellars"], dtype=np.float64)
        for i in range(data["stellars"]["nstellars"]):
            line = content[i+2].rstrip().split(",")
            for j, entry in enumerate(stellar_vars):
                # Try to convert to float
                try:
                    data["stellars"][entry][i] = np.float64(line[j])
                except ValueError:
                    data["stellars"][entry][i] = np.nan
        data["stellars"]["id"] = np.int32(data["stellars"]["id"])
except IOError:
    pass

return data


In [54]:
def get_info2(dump):
    
    info = SimpleNamespace()
    filename = "output_{dump:05d}/amr_{dump:05d}.out00001".format(dump=dump)
    
    with FortranFile(filename, 'r') as f:
        
        info.ncpu, = f.read_ints('i')
        info.ndim, = f.read_ints('i')
        nx, ny, nz = f.read_ints('i')
        info.nlevelmax, = f.read_ints('i')
        
        ngridmax, = f.read_ints('i')
        nboundary, = f.read_ints('i')
        ngrid_current, = f.read_ints('i')
        info.boxlen, = f.read_reals('f8')
        
        noutput, iout, ifout = f.read_ints('i')
        tout = f.read_reals('f8')
        aout = f.read_reals('f8')
        info.t, = f.read_reals('f8')
        dtold = f.read_reals('f8')
        dtnew = f.read_reals('f8')
        nstep,nstep_coarse = f.read_ints('i')
        einit, mass_tot_0, rho_tot = f.read_reals('f8')
        info.omega_m, info.omega_l, info.omega_k, info.omega_b, info.h0, aexp_ini,boxlen_ini = f.read_reals('f8')
        info.aexp, hexp, aexp_old, epot_tot_int, epot_tot_old = f.read_reals('f8')
        mass_sph, = f.read_reals('f8')
        
        headl = f.read_ints('i')
        taill = f.read_ints('i')
        numbl = f.read_ints('i')
        numbl = numbl.reshape(info.nlevelmax, info.ncpu)
        numbtot = f.read_ints('i')
        
        xbound = [0, 0, 0]
        if (nboundary > 0):
            headb = f.read_ints('i')
            tailb = f.read_ints('i')
            numbb = f.read_ints('i')
            numbb = numbb.reshape(nlevelmax, nboundary)
            xbound = [float(nx//2),float(ny//2),float(nz//2)]
        
        headf, tailf, numbf, used_mem, used_mem_tot = f.read_ints('i')
        
        ordering = f.read_ints("i")
        bound_key = f.read_ints("f8")
        info.bound_key = np.empty(info.ncpu+1)
        if(len(bound_key) != info.ncpu+1):
            print("Quad Hilbert not supported in python.")
            info.quadhilbert = True
        else:
            info.quadhilbert = False
            info.bound_key[:] = bound_key
        
    filename = "output_{dump:05d}/info_{dump:05d}.txt".format(dump=dump)
    data = ascii.read(filename, header_start=0, data_start=0, data_end=18, delimiter='=', names=["field", "value"])
    name = np.array(data["field"])
    val = np.array(data["value"])
    
    info.ordering, = val[np.where(name=="ordering type")]
    info.unit_l, = val[np.where(name=="unit_l")]
    info.unit_d, = val[np.where(name=="unit_d")]
    info.unit_t, = val[np.where(name=="unit_t")]
    
    return info


In [55]:
def hilbert3d(x, y, z, bitlen):
    
    state_diagram = [ 1, 2, 3, 2, 4, 5, 3, 5,
                      0, 1, 3, 2, 7, 6, 4, 5,
                      2, 6, 0, 7, 8, 8, 0, 7,
                      0, 7, 1, 6, 3, 4, 2, 5,
                      0, 9, 10, 9, 1, 1, 11, 11,
                      0, 3, 7, 4, 1, 2, 6, 5,
                      6, 0, 6, 11, 9, 0, 9, 8,
                      2, 3, 1, 0, 5, 4, 6, 7,
                      11,11, 0, 7, 5, 9, 0, 7,
                      4, 3, 5, 2, 7, 0, 6, 1,
                      4, 4, 8, 8, 0, 6,10, 6,
                      6, 5, 1, 2, 7, 4, 0, 3,
                      5, 7, 5, 3, 1, 1,11,11,
                      4, 7, 3, 0, 5, 6, 2, 1,
                      6, 1, 6, 10, 9, 4, 9, 10,
                      6, 7, 5, 4, 1, 0, 2, 3,
                      10, 3, 1, 1, 10, 3, 5, 9,
                      2, 5, 3, 4, 1, 6, 0, 7,
                      4, 4, 8, 8, 2, 7, 2, 3,
                      2, 1, 5, 6, 3, 0, 4, 7,
                      7, 2,11, 2, 7, 5, 8, 5,
                      4, 5, 7, 6, 3, 2, 0, 1,
                      10, 3, 2, 6, 10, 3, 4, 4,
                      6, 1, 7, 0, 5, 2, 4, 3]

    state_diagram = np.array(state_diagram)
    state_diagram = state_diagram.reshape((8, 2, 12), order='F')
    order = np.zeros_like(x)
    bit_mask = np.zeros(3*bitlen, dtype=bool)
    
    for i in range(len(x)):
        
        j = np.arange(bitlen)
        bit_mask[2::3] = x[i] & (1 << np.arange(bitlen))
        bit_mask[1::3] = y[i] & (1 << np.arange(bitlen))
        bit_mask[0::3] = z[i] & (1 << np.arange(bitlen))

        state = 0
        for j in range(bitlen-1, -1, -1):
            sdigit = np.sum(bit_mask[3*j:3*(j+1)] * 2**np.arange(3))
            bit_mask[3*j:3*(j+1)] = state_diagram[sdigit, 1, state] & (1 << np.arange(3))
            state = state_diagram[sdigit, 0, state]
            
        order[i] = np.sum(bit_mask * 2**np.arange(3*bitlen))

    return order


In [8]:
radius = 10*const.kpc
center = np.array([0, 0, 0])

ndim = 3
boxlen = info.aexp*100*const.Mpc/const.h0
radius_code = radius/info.length_unit
center_code = center/info.length_unit + np.array([0.5, 0.5, 0.5])

level_min = np.max([1, math.floor(np.log2(1/radius_code/2))])
level_max = info.amr_level_sim_max
bitlen = level_min-1

dkey = 2**(ndim*(level_max-level_min))
ibound = [0, 0, 0, 0, 0, 0]

if (bitlen > 0):
    ibound[0:3] = np.array((center_code-radius_code)*2**bitlen, dtype=int)
    ibound[3:6] = np.array((center_code+radius_code)*2**bitlen, dtype=int)
    ndom = 8
    idom = [ibound[0], ibound[3]]*4
    jdom = ([ibound[1]]*2 + [ibound[4]]*2)*2
    kdom = [ibound[2]]*4 + [ibound[5]]*4
    order_min = hilbert3d(idom, jdom, kdom, bitlen)
else:
    ndom = 1
    order_min = np.array([0.])
    
bounding_min = order_min*dkey
bounding_max = (order_min+1)*dkey

cpu_min = np.zeros(ndom, dtype=int)
cpu_max = np.zeros(ndom, dtype=int)
for idx_cpu in range(0, info.ncpu):
    for idom in range(0, ndom):
        if( (info.bound_key[icpu] <= bounding_min[idom]) and (info.bound_key[icpu+1] > bounding_min[idom]) ):
            cpu_min[idom] = icpu+1
        if( (info.bound_key[icpu] < bounding_max[idom]) and (info.bound_key[icpu+1] >= bounding_max[idom]) ):
            cpu_max[idom] = icpu+1


ncpu_read = 0
cpu_read = np.zeros(ncpu, dtype=bool)
cpu_list = []
for idom in range(0,ndom):
    for icpu in range(cpu_min[idom]-1,cpu_max[idom]):
        if ( not cpu_read[icpu] ):
            cpu_list.append(icpu+1)
            ncpu_read = ncpu_read+1
            cpu_read[icpu] = True


AttributeError: 'types.SimpleNamespace' object has no attribute 'bound_key'

In [285]:
def get_cpu_list(center, radius):
    
    for i in range(0, info.nlevelmax):
        dx = 1/2**ilevel
        if (dx < 2*radius/info.boxlen):
            break

    levelmin = np.max([ilevel,1])
    bit_length = levelmin-1
    nmax = 2**bit_length
    ndim = info.ndim
    ncpu = info.ncpu
    nlevelmax = info.nlevelmax
    dkey = 2**(ndim*(nlevelmax+1-bit_length))
    ibound = [0, 0, 0, 0, 0, 0]
    if(bit_length > 0):
        ibound[0:3] = (center-radius)*nmax/info.boxlen
        ibound[3:6] = (center+radius)*nmax/info.boxlen
        ibound[0:3] = np.array(ibound[0:3]).astype(int)
        ibound[3:6] = np.array(ibound[3:6]).astype(int)
        ndom = 8
        idom = [ibound[0], ibound[3], ibound[0], ibound[3], ibound[0], ibound[3], ibound[0], ibound[3]]
        jdom = [ibound[1], ibound[1], ibound[4], ibound[4], ibound[1], ibound[1], ibound[4], ibound[4]]
        kdom = [ibound[2], ibound[2], ibound[2], ibound[2], ibound[5], ibound[5], ibound[5], ibound[5]]
        order_min = hilbert3d(idom,jdom,kdom,bit_length)
    else:
        ndom = 1
        order_min = np.array([0.])
        
    bounding_min = order_min*dkey
    bounding_max = (order_min+1)*dkey

    cpu_min = np.zeros(ndom, dtype=int)
    cpu_max = np.zeros(ndom, dtype=int)
    for icpu in range(0,ncpu):
        for idom in range(0,ndom):
            if( (info.bound_key[icpu] <= bounding_min[idom]) and (info.bound_key[icpu+1] > bounding_min[idom]) ):
                cpu_min[idom] = icpu+1
            if( (info.bound_key[icpu] < bounding_max[idom]) and (info.bound_key[icpu+1] >= bounding_max[idom]) ):
                cpu_max[idom] = icpu+1


    ncpu_read = 0
    cpu_read = np.zeros(ncpu, dtype=bool)
    cpu_list = []
    for idom in range(0,ndom):
        for icpu in range(cpu_min[idom]-1,cpu_max[idom]):
            if ( not cpu_read[icpu] ):
                cpu_list.append(icpu+1)
                ncpu_read = ncpu_read+1
                cpu_read[icpu] = True

    return cpu_list


In [59]:
get_cpu_list(0, 10)


AttributeError: 'types.SimpleNamespace' object has no attribute 'nlevelmax'

In [26]:
id_star = np.array([])
    tau_starbirth = np.array([])
    for i in range(1, info.ncpu+1):
        partfile = os.path.join("output_%.5d" % dump, "part_%.5d.out%.5d" % (dump, i))
        with FortranFile(partfile, 'r') as f:
            _, _, _ = f.read_ints('i'), f.read_ints('i'), f.read_ints('i')
            _, _, _, _, _ = f.read_reals('f8'), f.read_reals('f4'), f.read_reals('f8'), f.read_reals('f8'), f.read_reals('f4')
            _, _, _ = f.read_reals('f8'), f.read_reals('f8'), f.read_reals('f8') # position
            _, _, _ = f.read_reals('f8'), f.read_reals('f8'), f.read_reals('f8') # velocity
            _ = f.read_reals('f8') # mass
            id_part = f.read_ints('i') # id
            _ = f.read_ints('i') # level
            type_part = f.read_ints('b') # family
            _ = f.read_ints('b') # tag
            tau_partbirth = f.read_reals('f8') # birthtime
            _ = f.read_reals('f8') # metallicity
            id_star = np.concatenate((id_star, id_part[type_part==STAR]))
            tau_starbirth = np.concatenate((tau_starbirth, tau_partbirth[type_part==STAR]))
    integrand = lambda a: (info.Omega_m0 * a**(-1) + info.Omega_k0 + info.Omega_L0 * a**2)**(-1/2)
    age_universe = quad(integrand, 0, 1)[0] / info.H0
    time_starbirth = tau_starbirth / info.H0 + age_universe
    return id_star, time_starbirth


In [31]:
read_partfile(54)


NameError: name 'quad' is not defined